## Import Libraries and Load the Data 

In [1]:
import numpy as np 
from keras.models import Sequential 
from keras.layers import Dense, Activation, Dropout 
from keras.optimizers import SGD 
import random 

import nltk 
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json 
import pickle 

intents_file = open(r'C:\Users\akobe\OneDrive\Desktop\Lighthouse\After\Chatbot\Data\intents.json').read()
intents = json.loads(intents_file)

In [5]:
#pip install numpy==1.21.1
#used different kernel to install different version of numpy 

## Preprocessing Data 

In [2]:
#preprocessing data 
words = []
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']

In [3]:
#preprocessing data 
for intent in intents['intents']:
    for pattern in intent['patterns']: 
        #tokenize each word 
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        #add documents in corpus 
        documents.append((word, intent['tag']))
        #add to our classes list  
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

print(documents)

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['Open', 'adverse', 'drugs', 'module'], 'adverse_drug'), (['Give', 'm

In [4]:
#lemmatization 
# lemmatize and lower each wor and remove duplicates 
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))

#sort classes 
calsses = sorted(list(set(classes)))

#documents = combination between patterns and intents 
print(len(documents), 'documents')
#classes = intents 
print(len(classes), 'classes', classes)
#words = all words, vocabulary 
print(len(words), 'unique lemmatized words', words)

#pickle.dump(words,open('words.pkl','wb'))
#pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['greeting', 'goodbye', 'thanks', 'options', 'adverse_drug', 'blood_pressure', 'blood_pressure_search', 'pharmacy_search', 'hospital_search']
87 unique lemmatized words ["'s", 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


## Create Training and Testing Data 

In [5]:
#create the training data 
training = []

#create empty array for the output 
output_empty = [0]* len(classes)

#training set, bag of words for every sentence 
for doc in documents: 
    #initializing bag of words 
    bag = []
    #list of toeknized words for the pattern 
    word_patterns = doc[0]
    
    #lemmatize each word - create base wordm in attempt to represent related words 
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    
    #create bag of words array with 1, if word is found in current pattern 
    for word in words: 
        bag.append(1) if word in word_patterns else bag.append(0)
    
    #outpur is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

#shuffle the features and make a numpy array 
random.shuffle(training)
training = np.array(training)

#create training and testing lists. X - patterns, Y-intents 
train_x = list(training[:, 0])
train_y = list(training[:, 1])

print("Training data is created")

Training data is created


C:\Users\akobe\AppData\Local\Temp\ipykernel_9764\3047850909.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


## Training the Model

In [10]:
#deep neural networks model 
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

#compliling model. SGD with  Nesterov acelrated gradient gives good results of this model 
sgd = SGD(lr=0.01, decay = 1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#trainin and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model is created")

Epoch 1/200
10/10 [==============================] - 6s 7ms/step - loss: 2.1951 - accuracy: 0.1702 
Epoch 2/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1573 - accuracy: 0.1489
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0609 - accuracy: 0.2766
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 2.0219 - accuracy: 0.2766
Epoch 5/200
10/10 [==============================] - 0s 3ms/step - loss: 1.8518 - accuracy: 0.4043
Epoch 6/200
10/10 [==============================] - 0s 3ms/step - loss: 1.7665 - accuracy: 0.4681
Epoch 7/200
10/10 [==============================] - 0s 4ms/step - loss: 1.6424 - accuracy: 0.4043
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.6079 - accuracy: 0.4681
Epoch 9/200
10/10 [==============================] - 0s 3ms/step - loss: 1.3723 - accuracy: 0.6383
Epoch 10/200
10/10 [==============================] - 0s 3ms/step - loss: 1.3456 - accuracy: 0.5957
Epoch 11

## Interacting with the Chatbot

##### code - will also create a py. file 

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model

model = load_model('chatbot_model.h5')

import json
import random

